In [1]:
import csv

def get_csv_schema(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Get the header row
        data_types = []
        for i, column_name in enumerate(header):
            try:
                # Attempt to convert data in the first row to int, float, etc.
                first_row = next(reader)
                _ = int(first_row[i])
                data_types.append("int")
            except ValueError:
                try:
                    _ = float(first_row[i])
                    data_types.append("float")
                except ValueError:
                     data_types.append("string")
            finally:
                file.seek(0)
                next(reader) # Reset reader to the second row for next use
        return dict(zip(header, data_types))

# Example usage:
# "C:\\Users\\scott\\Downloads\\MOCK_DATA.csv"
file_path = 'C:\\Users\\scott\\Downloads\\MOCK_DATA.csv'
schema = get_csv_schema(file_path)
print(schema)

{'id': 'int', 'first_name': 'string', 'last_name': 'string', 'email': 'string', 'gender': 'string', 'ip_address': 'string'}
